In [0]:
apt_Data = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.layer_27240
""")
display(apt_Data)

In [0]:
from pyspark.sql import functions as F

def convert_to_unsigned(layer_id, high, low):
    # layer_id = 14533
    unsigned_high = int(high) & ((1 << 64) - 1)
    unsigned_low = int(low) & ((1 << 64) - 1)
    return f"{layer_id}:{unsigned_high}:{unsigned_low}"


# Assuming df is your DataFrame
apt_Data_with_unsigned_id = apt_Data.withColumn("unsigned_id", F.udf(convert_to_unsigned, "string")("id.layerId","id.high", "id.low"))

display(apt_Data_with_unsigned_id)

In [0]:

# get country name as coulumn
apt_Data_with_unsigned_id = apt_Data_with_unsigned_id.withColumn("country_name", F.expr("case when exists(tags, x -> x.tagKey.key = 'metadata:country') then filter(tags, x -> x.tagKey.key = 'metadata:country')[0].value else null end")).withColumn("state_name", F.expr("case when exists(tags, x -> x.tagKey.key = 'addr:state:en-Latn') then filter(tags, x -> x.tagKey.key = 'addr:state:en-Latn')[0].value else null end"))
display(apt_Data_with_unsigned_id)

In [0]:
apt_Data_with_unsigned_id.groupBy("country_name", "state_name").count().orderBy("count", ascending=False).display()

In [0]:
from pyspark.sql import functions as F

# filter based on country and state_name
apt_Data = apt_Data_with_unsigned_id.filter("country_name = 'USA'").filter("state_name = 'IL'")

# .filter(
#     F.expr("exists(tags, x -> x.tagKey.key = 'metadata:country' AND x.value = 'AUT')")
#)
# .filter(
#     F.expr("exists(tags, x -> x.tagKey.key = 'metadata:source:name' AND x.value like '%Geolytica%')")
# )
# .filter(
#     F.expr("exists(tags, x -> x.tagKey.key = 'metadata:source:id' AND x.value = 'Du6RYnGNtcRZFuyf03wib')")
# )
print(f"count: {apt_Data.count()}")
# display(apt_Data_GBR)

In [0]:
# apt_Data_GBR_source = apt_Data_GBR.filter(
#     F.expr("exists(tags, x -> x.tagKey.key = 'metadata:source:id' AND (x.value like '%Du6RYnGNtcRZFuyf03wib%' OR x.value like '%aqt88mW-gcIkgRFs7lz73%'))")
# )
# print(f"GBR source count: {apt_Data_GBR_source.count()}")
# display(apt_Data_GBR_source)

In [0]:
# distinct_values = apt_Data.selectExpr("explode(tags) as tag") \
#                    .filter("tag.tagKey.key = 'metadata:source:id'") \
#                    .select("tag.value") \
#                    .distinct()

# # Show the distinct values
# display(distinct_values)

In [0]:
dbutils.fs.mounts()

In [0]:
dbutils.fs.unmount("/mnt/layers/19174/snapshot")

In [0]:
directory_path = "/mnt/opas/opas-source"

files = dbutils.fs.ls(directory_path)
for file in files:
    print(file.name)

In [0]:
# write dataset in csv format
apt_Data.select("unsigned_id").coalesce(1).write.mode("overwrite").csv("/mnt/opas/opas-source/dev-apt-to-be-deleted/delete_usa_il")